In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers,losses,layers
import pickle
from glob import glob
from os.path import basename
import networkx as nx
import random

In [2]:
dataset = 'AIDS'

In [3]:
def load_generated_graphs(dataset_name, file_name='generated_graph_500'):

    dir = './dataset/' + dataset_name + '/' + file_name
    g = open(dir, 'rb')
    generated_graphs = pickle.load(g)
    g.close()
    return generated_graphs

training_pairs = load_generated_graphs(dataset, file_name='generated_graph_pairs')
test_gt_ged = load_generated_graphs(dataset, file_name='ged_matrix_test')

In [5]:
g = open('./dataset/' + dataset + '/AIDS', 'rb')
all_graphs = pickle.load(g)
g.close()

In [6]:
class MyModel(tf.keras.Model):

    def __init__(self):
        super().__init__()
        self.input_nc = tf.keras.Input(shape=(16,))
        self.fully_connected_first_nc = tf.keras.layers.Dense(16, activation="relu")
        self.fully_connected_second_nc = tf.keras.layers.Dense(8, activation="relu")
        self.fully_connected_third_nc = tf.keras.layers.Dense(4, activation="relu")
        self.scoring_layer_nc = tf.keras.layers.Dense(1, activation="sigmoid")

        self.scoring_layer_nc = keras.Sequential([
            self.input_nc, 
            self.fully_connected_first_nc,
            self.fully_connected_second_nc,
            self.fully_connected_third_nc,
            self.scoring_layer_nc])
    
    
        self.input_in = tf.keras.Input(shape=(16,))
        self.fully_connected_first_in = tf.keras.layers.Dense(16, activation="relu")
        self.fully_connected_second_in = tf.keras.layers.Dense(8, activation="relu")
        self.fully_connected_third_in = tf.keras.layers.Dense(4, activation="relu")
        self.scoring_layer_in = tf.keras.layers.Dense(1, activation="sigmoid")

        self.scoring_layer_in = keras.Sequential([
            self.input_in, ##feature_count = tensor_neurons = 16
            self.fully_connected_first_in,
            self.fully_connected_second_in,
            self.fully_connected_third_in,
            self.scoring_layer_in])

        
        self.input_ie = tf.keras.Input(shape=(16,))
        self.fully_connected_first_ie = tf.keras.layers.Dense(16, activation="relu")
        self.fully_connected_second_ie = tf.keras.layers.Dense(8, activation="relu")
        self.fully_connected_third_ie = tf.keras.layers.Dense(4, activation="relu")
        self.scoring_layer_ie = tf.keras.layers.Dense(1, activation="sigmoid")

        self.scoring_layer_ie = keras.Sequential([
            self.input_ie, ##feature_count = tensor_neurons = 16
            self.fully_connected_first_ie,
            self.fully_connected_second_ie,
            self.fully_connected_third_ie,
            self.scoring_layer_ie])
        
        
        self.input_ec = tf.keras.Input(shape=(16,))
        self.fully_connected_first_ec = tf.keras.layers.Dense(16, activation="relu")
        self.fully_connected_second_ec = tf.keras.layers.Dense(8, activation="relu")
        self.fully_connected_third_ec = tf.keras.layers.Dense(4, activation="relu")
        self.scoring_layer_ec = tf.keras.layers.Dense(1, activation="sigmoid")

        self.scoring_layer_ec = keras.Sequential([
            self.input_ec, ##feature_count = tensor_neurons = 16
            self.fully_connected_first_ec,
            self.fully_connected_second_ec,
            self.fully_connected_third_ec,
            self.scoring_layer_ec])

        
        g = open('./dataset/' + dataset + '/global_node_label', 'rb')
        global_node_labels = pickle.load(g)
        g.close()

        g = open('./dataset/' + dataset + '/global_edge_label', 'rb')
        global_edge_labels = pickle.load(g)
        g.close()

        number_of_node_labels = len(global_node_labels)
        number_of_edge_labels = len(global_edge_labels)
        self.input_dim_node = 2*number_of_node_labels

        initializer1 = tf.keras.initializers.GlorotUniform()
        self.weight_matrix_node = initializer1(shape = (self.input_dim_node, self.input_dim_node, 16), dtype = tf.float32)
        self.weight_matrix_node_block = initializer1(shape= (16, 2*self.input_dim_node), dtype = tf.float32)
        self.bias_node = initializer1(shape = (16, 1), dtype = tf.float32)
        
        self.input_dim_edge = 2*number_of_edge_labels

        initializer2 = tf.keras.initializers.GlorotUniform()
        self.weight_matrix_edge = initializer2(shape = (self.input_dim_edge, self.input_dim_edge, 16), dtype = tf.float32)
        self.weight_matrix_edge_block = initializer2(shape= (16, 2*self.input_dim_edge), dtype = tf.float32)
        self.bias_edge = initializer2(shape = (16, 1), dtype = tf.float32)


    def call(self, data):
    
        adj_1 = tf.constant(np.array(data["edge_index_1"].todense()),dtype=tf.float32)
        adj_2 = tf.constant(np.array(data["edge_index_2"].todense()),dtype=tf.float32)
        edge_adj_1, edge_adj_2 =  data["edge_adj_1"], data["edge_adj_2"]
        features_1, features_2 = data["features_1"], data["features_2"]
        edge_features_1, edge_features_2 = data["edge_features_1"], data["edge_features_2"]


        #GAL 
        graph1_hidden1 = tf.matmul( adj_1,features_1)
        graph1_hidden2 = tf.matmul( adj_1,graph1_hidden1)

        graph2_hidden1 = tf.matmul(adj_2,features_2)
        graph2_hidden2 = tf.matmul(adj_2,graph2_hidden1)
        
        edge1_hidden1 = tf.matmul( edge_adj_1,edge_features_1)
        edge1_hidden2 = tf.matmul( edge_adj_1,edge1_hidden1)

        edge2_hidden1 = tf.matmul( edge_adj_2,edge_features_2)
        edge2_hidden2 = tf.matmul( edge_adj_2,edge2_hidden1)

        graph1_01concat = tf.concat([features_1, graph1_hidden1], axis=1)
        graph2_01concat = tf.concat([features_2, graph2_hidden1], axis=1)
        graph1_12concat = tf.concat([graph1_hidden1, graph1_hidden2], axis=1)
        graph2_12concat = tf.concat([graph2_hidden1, graph2_hidden2], axis=1)

        graph1_01pooled = tf.expand_dims(tf.reduce_sum(graph1_01concat,axis=0),1)
        graph1_12pooled = tf.expand_dims(tf.reduce_sum(graph1_12concat,axis=0),1)
        graph2_01pooled = tf.expand_dims(tf.reduce_sum(graph2_01concat,axis=0),1)
        graph2_12pooled = tf.expand_dims(tf.reduce_sum(graph2_12concat,axis=0),1)
        
        edge1_01concat = tf.concat([edge_features_1, edge1_hidden1], axis=1)
        edge2_01concat = tf.concat([edge_features_2, edge2_hidden1], axis=1)

        edge1_01pooled = tf.expand_dims(tf.reduce_sum(edge1_01concat,axis=0),1)#sum
        edge2_01pooled = tf.expand_dims(tf.reduce_sum(edge2_01concat,axis=0),1)
        


        scoring = tf.matmul(tf.transpose(graph1_01pooled, perm=[1,0]), self.weight_matrix_node.reshape(self.input_dim_node, -1))
        scoring = scoring.reshape(self.input_dim_node, 16)
        scoring = tf.matmul(tf.transpose(scoring, perm=[1,0]), graph2_01pooled)
        combined_representation = tf.concat((graph1_01pooled, graph2_01pooled), axis=0)
        block_scoring = tf.matmul(self.weight_matrix_node_block, combined_representation)
        scores_nc = tf.nn.relu(scoring + block_scoring + self.bias_node)
        scores_nc = tf.transpose(scores_nc, perm=[1,0])


        scores_nc = self.scoring_layer_nc(scores_nc)
        #print("score_nc: \n", scores_nc)

        scoring = tf.matmul(tf.transpose(graph1_01pooled, perm=[1,0]), self.weight_matrix_node.reshape(self.input_dim_node, -1))
        scoring = scoring.reshape(self.input_dim_node, 16)
        scoring = tf.matmul(tf.transpose(scoring, perm=[1,0]), graph2_01pooled)
        combined_representation = tf.concat((graph1_01pooled, graph2_01pooled),axis=0)
        block_scoring = tf.matmul(self.weight_matrix_node_block, combined_representation)
        scores_in = tf.nn.relu(scoring + block_scoring + self.bias_node)
        scores_in = tf.transpose(scores_in, perm=[1,0])


        scores_in = self.scoring_layer_in(scores_in)
        #print("score_in: \n", scores_in)


        scoring = tf.matmul(tf.transpose(graph1_12pooled, perm=[1,0]), self.weight_matrix_node.reshape(self.input_dim_node, -1))
        scoring = scoring.reshape(self.input_dim_node, 16)
        scoring = tf.matmul(tf.transpose(scoring, perm=[1,0]), graph2_12pooled)
        combined_representation = tf.concat((graph1_12pooled, graph2_12pooled),axis = 0)
        block_scoring = tf.matmul(self.weight_matrix_node_block, combined_representation)
        scores_ie = tf.nn.relu(scoring + block_scoring + self.bias_node)
        scores_ie = tf.transpose(scores_ie, perm=[1,0])


        scores_ie = self.scoring_layer_ie(scores_ie)
        #print("score_ie: \n", scores_ie)
        
        
        scoring = tf.matmul(tf.transpose(edge1_01pooled, perm=[1,0]), self.weight_matrix_edge.reshape(self.input_dim_edge, -1))
        scoring = scoring.reshape(self.input_dim_edge, 16)
        scoring = tf.matmul(tf.transpose(scoring, perm=[1,0]), edge2_01pooled)
        combined_representation = tf.concat((edge1_01pooled, edge2_01pooled),axis=0)
        block_scoring = tf.matmul(self.weight_matrix_edge_block, combined_representation)
        scores_ec = tf.nn.relu(scoring + block_scoring + self.bias_edge)
        scores_ec = tf.transpose(scores_ec, perm=[1,0])


        scores_ec = self.scoring_layer_ec(scores_ec)
        
        return(tf.concat([scores_nc, scores_in, scores_ie, scores_ec], axis=1))

In [7]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
def transfer_to_tensorflow(data, type_specified=True):
 
    g = open('./dataset/' + dataset + '/global_node_label', 'rb')
    global_node_labels = pickle.load(g)
    g = open('./dataset/' + dataset + '/global_edge_label', 'rb')
    global_edge_labels = pickle.load(g)
    
    new_data = dict()
    graph1, graph2 = data['graph_pair'][0], data['graph_pair'][1]
    nodes1, nodes2 = list(graph1.nodes()), list(graph2.nodes())

    features_1, features_2, edge_features_1, edge_features_2, edge_adj_1, edge_adj_2 = [], [], [], [], [], []

    for n in graph1.nodes():
        features_1.append([1.0 if graph1.nodes()[n]['type'] == ele else 0.0 for ele in global_node_labels])

    for n in graph2.nodes():
        features_2.append([1.0 if graph2.nodes()[n]['type'] == ele else 0.0 for ele in global_node_labels])
    features_1, features_2 = tf.constant(np.array(features_1),dtype = tf.float32), tf.constant(np.array(features_2),dtype = tf.float32)

    for e in graph1.edges():
        edge_features_1.append([1.0 if graph1.edges()[e]['type'] == ele else 0.0 for ele in global_edge_labels])
        adj_row = []
        for d in graph1.edges():
            if(e == d):
                adj_row.append(0.0)
                continue
            if((e[0] in d) | (e[1] in d)):
                adj_row.append(1.0)
            else:
                adj_row.append(0.0)
        edge_adj_1.append(adj_row)

    for e in graph2.edges():
        edge_features_2.append([1.0 if graph2.edges()[e]['type'] == ele else 0.0 for ele in global_edge_labels])
        adj_row = []
        for d in graph2.edges():
            if(e == d):
                adj_row.append(0.0)
                continue
            if((e[0] in d) | (e[1] in d)):
                adj_row.append(1.0)
            else:
                adj_row.append(0.0)
        edge_adj_2.append(adj_row)

    edge_features_1 = tf.constant(np.array(edge_features_1),dtype = tf.float32)
    edge_features_2 = tf.constant(np.array(edge_features_2),dtype = tf.float32)
    edge_adj_1, edge_adj_2 = tf.constant(np.array(edge_adj_1),dtype = tf.float32), tf.constant(np.array(edge_adj_2),dtype = tf.float32)

    new_data["edge_index_1"], new_data["edge_index_2"] = nx.adjacency_matrix(graph1), nx.adjacency_matrix(graph2)
    new_data["features_1"], new_data["features_2"] = features_1, features_2
    new_data["edge_features_1"], new_data["edge_features_2"] = edge_features_1, edge_features_2
    new_data["edge_adj_1"], new_data["edge_adj_2"] = edge_adj_1, edge_adj_2


    if(type_specified):
        norm_ged = [data['ged'][key] / (0.5 * (graph1.number_of_nodes() + graph2.number_of_nodes())) for key in ['nc', 'in', 'ie', 'ec']]
        norm_ged = np.array(norm_ged)
        new_data["target"] = tf.cast(tf.convert_to_tensor(np.exp(-norm_ged)).reshape(1, -1), tf.float32)
            
        norm_gt_ged = (data['ged']['nc'] + data['ged']['in'] + data['ged']['ie'] + data['ged']['ec']) / (0.5 * (graph1.number_of_nodes() + graph2.number_of_nodes()))
        new_data["gt_ged"] = tf.cast(tf.convert_to_tensor(np.exp(-norm_gt_ged).reshape(1, 1)).reshape(1, -1),tf.float32)
    else:
        norm_gt_ged = data['ged'] / (0.5 * (graph1.number_of_nodes() + graph2.number_of_nodes()))
        new_data["gt_ged"] = tf.cast(tf.convert_to_tensor(np.exp(-norm_gt_ged).reshape(1, 1)).reshape(1, -1),tf.float32)

    return new_data

In [8]:
batch_size = 128
epochs = 1
mse = tf.keras.losses.MeanSquaredError()
optimizer = keras.optimizers.Adam(learning_rate = 1e-3)
model = MyModel()

for epoch in range(epochs):
    random.shuffle(training_pairs)
    batches = []
    for graph in range(0, len(training_pairs), batch_size):
          batches.append(training_pairs[graph:graph+batch_size])
    iteration = 0
    for batch in batches:

        losses = 0  
        with tf.GradientTape() as tape:
            for graph_pair in batch:
                data = transfer_to_tensorflow(graph_pair)
                model.compile(optimizer=optimizer, loss=mse)
                test = model(data)
                losses += mse(data["target"], test)

        grads = tape.gradient(losses, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        loss = losses.numpy()    
        print ("Iteration: ", iteration, "loss: ", loss/len(batch))  
        iteration+=1

        
test_scores = []
for i in range(40200, len(all_graphs)):
    for j in range(len(all_graphs)):
        if((i, j) in test_gt_ged):
            curr_graph_pair = {'graph_pair': [all_graphs[i], all_graphs[j]], 'ged':test_gt_ged[(i, j)]}
            data = transfer_to_tensorflow(curr_graph_pair, type_specified=False)
            prediction = model(data)
            prediction = tf.math.exp(tf.math.reduce_sum(tf.math.log(prediction))).reshape(1, -1)
            current_error = mse(prediction, data["gt_ged"])

            test_scores.append(current_error)


model_error = sum(test_scores) / len(test_scores)
print("\nModel test error: " + str(model_error))

C:\Users\huang\AppData\Local\Temp\ipykernel_12932\4068969007.py:53: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  new_data["edge_index_1"], new_data["edge_index_2"] = nx.adjacency_matrix(graph1), nx.adjacency_matrix(graph2)


Iteration:  0 loss:  0.5047422647476196
Iteration:  1 loss:  0.5003044009208679
Iteration:  2 loss:  0.4895554482936859
Iteration:  3 loss:  0.4624481201171875
Iteration:  4 loss:  0.4637759327888489
Iteration:  5 loss:  0.4642687141895294
Iteration:  6 loss:  0.45162737369537354
Iteration:  7 loss:  0.4286254644393921
Iteration:  8 loss:  0.42863142490386963
Iteration:  9 loss:  0.4154314696788788
Iteration:  10 loss:  0.414191335439682
Iteration:  11 loss:  0.4121845066547394
Iteration:  12 loss:  0.406529039144516
Iteration:  13 loss:  0.40194591879844666
Iteration:  14 loss:  0.41083890199661255
Iteration:  15 loss:  0.3957781195640564
Iteration:  16 loss:  0.3901534676551819
Iteration:  17 loss:  0.3914688527584076
Iteration:  18 loss:  0.3834851384162903
Iteration:  19 loss:  0.37712550163269043
Iteration:  20 loss:  0.39363667368888855
Iteration:  21 loss:  0.3745403289794922
Iteration:  22 loss:  0.3631660044193268
Iteration:  23 loss:  0.36025315523147583
Iteration:  24 loss: 

Iteration:  194 loss:  0.026780663058161736
Iteration:  195 loss:  0.02922779694199562
Iteration:  196 loss:  0.02905264124274254
Iteration:  197 loss:  0.028604907914996147
Iteration:  198 loss:  0.029226627200841904
Iteration:  199 loss:  0.02853553369641304
Iteration:  200 loss:  0.029020145535469055
Iteration:  201 loss:  0.02734425477683544
Iteration:  202 loss:  0.026433315128087997
Iteration:  203 loss:  0.027450088411569595
Iteration:  204 loss:  0.026711780577898026
Iteration:  205 loss:  0.027901770547032356
Iteration:  206 loss:  0.028137048706412315
Iteration:  207 loss:  0.025613535195589066
Iteration:  208 loss:  0.02802960015833378
Iteration:  209 loss:  0.02796969562768936
Iteration:  210 loss:  0.027848314493894577
Iteration:  211 loss:  0.025832490995526314
Iteration:  212 loss:  0.02537817507982254
Iteration:  213 loss:  0.02757451869547367
Iteration:  214 loss:  0.02695152722299099
Iteration:  215 loss:  0.02732086181640625
Iteration:  216 loss:  0.02788724936544895

Iteration:  382 loss:  0.01934376172721386
Iteration:  383 loss:  0.021658824756741524
Iteration:  384 loss:  0.019867710769176483
Iteration:  385 loss:  0.02154180221259594
Iteration:  386 loss:  0.02121436595916748
Iteration:  387 loss:  0.0209079310297966
Iteration:  388 loss:  0.020411381497979164
Iteration:  389 loss:  0.019912036135792732
Iteration:  390 loss:  0.01995399408042431
Iteration:  391 loss:  0.019833702594041824
Iteration:  392 loss:  0.01994018256664276
Iteration:  393 loss:  0.021744277328252792
Iteration:  394 loss:  0.021731501445174217
Iteration:  395 loss:  0.019565638154745102
Iteration:  396 loss:  0.019779106602072716
Iteration:  397 loss:  0.018821626901626587
Iteration:  398 loss:  0.019894585013389587
Iteration:  399 loss:  0.02112390287220478
Iteration:  400 loss:  0.02071768417954445
Iteration:  401 loss:  0.0191878080368042
Iteration:  402 loss:  0.01978696882724762
Iteration:  403 loss:  0.021182004362344742
Iteration:  404 loss:  0.020079558715224266


Iteration:  570 loss:  0.01591462828218937
Iteration:  571 loss:  0.017292151227593422
Iteration:  572 loss:  0.01773407682776451
Iteration:  573 loss:  0.01613699272274971
Iteration:  574 loss:  0.01664518192410469
Iteration:  575 loss:  0.017752302810549736
Iteration:  576 loss:  0.016108190640807152
Iteration:  577 loss:  0.017461959272623062
Iteration:  578 loss:  0.015597619116306305
Iteration:  579 loss:  0.01527052465826273
Iteration:  580 loss:  0.016764400526881218
Iteration:  581 loss:  0.01761285774409771
Iteration:  582 loss:  0.017198417335748672
Iteration:  583 loss:  0.01528894528746605
Iteration:  584 loss:  0.01875876449048519
Iteration:  585 loss:  0.015954483300447464
Iteration:  586 loss:  0.01523321121931076
Iteration:  587 loss:  0.015508173033595085
Iteration:  588 loss:  0.015854107216000557
Iteration:  589 loss:  0.015127155929803848
Iteration:  590 loss:  0.015676945447921753
Iteration:  591 loss:  0.016855865716934204
Iteration:  592 loss:  0.0144640514627099

...

Iteration:  4586 loss:  0.003490266390144825
Iteration:  4587 loss:  0.004390187561511993
Iteration:  4588 loss:  0.0033009599428623915
Iteration:  4589 loss:  0.003709511598572135
Iteration:  4590 loss:  0.003939371090382338
Iteration:  4591 loss:  0.004654320422559977
Iteration:  4592 loss:  0.0034919234458357096
Iteration:  4593 loss:  0.004135871306061745
Iteration:  4594 loss:  0.003741492982953787
Iteration:  4595 loss:  0.003998559899628162
Iteration:  4596 loss:  0.004225905053317547
Iteration:  4597 loss:  0.0037472955882549286
Iteration:  4598 loss:  0.004097265657037497
Iteration:  4599 loss:  0.003818714525550604
Iteration:  4600 loss:  0.004308654926717281
Iteration:  4601 loss:  0.00414159893989563
Iteration:  4602 loss:  0.0037538439501076937
Iteration:  4603 loss:  0.004504855256527662
Iteration:  4604 loss:  0.004229253623634577
Iteration:  4605 loss:  0.004117222502827644
Iteration:  4606 loss:  0.0035457091871649027
Iteration:  4607 loss:  0.0038019935600459576
Itera

Iteration:  4767 loss:  0.003692950587719679
Iteration:  4768 loss:  0.0045099081471562386
Iteration:  4769 loss:  0.003817599732428789
Iteration:  4770 loss:  0.0038626797031611204
Iteration:  4771 loss:  0.003514015581458807
Iteration:  4772 loss:  0.0040718307718634605
Iteration:  4773 loss:  0.0035256645642220974
Iteration:  4774 loss:  0.004262665752321482
Iteration:  4775 loss:  0.003983845468610525
Iteration:  4776 loss:  0.003584010526537895
Iteration:  4777 loss:  0.0037537419702857733
Iteration:  4778 loss:  0.004260602407157421
Iteration:  4779 loss:  0.0034742902498692274
Iteration:  4780 loss:  0.004195420537143946
Iteration:  4781 loss:  0.004006166942417622
Iteration:  4782 loss:  0.004175352863967419
Iteration:  4783 loss:  0.004055678844451904
Iteration:  4784 loss:  0.004037088248878717
Iteration:  4785 loss:  0.0040628123097121716
Iteration:  4786 loss:  0.004025146830826998
Iteration:  4787 loss:  0.004004159942269325
Iteration:  4788 loss:  0.0035985473077744246
It

Iteration:  4948 loss:  0.0036929443012923002
Iteration:  4949 loss:  0.0036181090399622917
Iteration:  4950 loss:  0.003647915553301573
Iteration:  4951 loss:  0.004032002296298742
Iteration:  4952 loss:  0.004136858507990837
Iteration:  4953 loss:  0.0037827882915735245
Iteration:  4954 loss:  0.004127641674131155
Iteration:  4955 loss:  0.004189193714410067
Iteration:  4956 loss:  0.0035698721185326576
Iteration:  4957 loss:  0.0038928959984332323
Iteration:  4958 loss:  0.004000669810920954
Iteration:  4959 loss:  0.00397864542901516
Iteration:  4960 loss:  0.00369730475358665
Iteration:  4961 loss:  0.0035170952323824167
Iteration:  4962 loss:  0.0039726789109408855
Iteration:  4963 loss:  0.003972678445279598
Iteration:  4964 loss:  0.0041321441531181335
Iteration:  4965 loss:  0.0037177042104303837
Iteration:  4966 loss:  0.0036322767846286297
Iteration:  4967 loss:  0.0035071384627372026
Iteration:  4968 loss:  0.0038209569174796343
Iteration:  4969 loss:  0.003762432374060154


Iteration:  5129 loss:  0.003419179003685713
Iteration:  5130 loss:  0.004039749037474394
Iteration:  5131 loss:  0.003592304652556777
Iteration:  5132 loss:  0.004232670180499554
Iteration:  5133 loss:  0.003491742769256234
Iteration:  5134 loss:  0.0033226783853024244
Iteration:  5135 loss:  0.00369381345808506
Iteration:  5136 loss:  0.0042770421132445335
Iteration:  5137 loss:  0.003799856174737215
Iteration:  5138 loss:  0.004027195740491152
Iteration:  5139 loss:  0.00402745371684432
Iteration:  5140 loss:  0.004099009092897177
Iteration:  5141 loss:  0.0040039182640612125
Iteration:  5142 loss:  0.004081488586962223
Iteration:  5143 loss:  0.003465798683464527
Iteration:  5144 loss:  0.0037358668632805347
Iteration:  5145 loss:  0.0046113585121929646
Iteration:  5146 loss:  0.00418493989855051
Iteration:  5147 loss:  0.00380355934612453
Iteration:  5148 loss:  0.004093223251402378
Iteration:  5149 loss:  0.004126305691897869
Iteration:  5150 loss:  0.003498224774375558
Iteration

Iteration:  5310 loss:  0.0039056919049471617
Iteration:  5311 loss:  0.0037092608399689198
Iteration:  5312 loss:  0.00395321287214756
Iteration:  5313 loss:  0.00409125629812479
Iteration:  5314 loss:  0.00385523634031415
Iteration:  5315 loss:  0.003484675893560052
Iteration:  5316 loss:  0.004230432212352753
Iteration:  5317 loss:  0.004654628224670887
Iteration:  5318 loss:  0.0038321209140121937
Iteration:  5319 loss:  0.003563141683116555
Iteration:  5320 loss:  0.003901189425960183
Iteration:  5321 loss:  0.003950043581426144
Iteration:  5322 loss:  0.0037376554682850838
Iteration:  5323 loss:  0.003979349043220282
Iteration:  5324 loss:  0.004301310982555151
Iteration:  5325 loss:  0.003637240966781974
Iteration:  5326 loss:  0.003614876652136445
Iteration:  5327 loss:  0.0038110818713903427
Iteration:  5328 loss:  0.0034577026963233948
Iteration:  5329 loss:  0.0037052729167044163
Iteration:  5330 loss:  0.0032313447445631027
Iteration:  5331 loss:  0.004051920957863331
Itera